In [50]:
import pandas as pd
import numpy as np
from pymongo import MongoClient

In [51]:
client = MongoClient('localhost', 27017)
db = client["MDAProjectDatabase"]

## Cases and deaths
- by day and county
- prepared by Aleksanda

### CSV to Parquet conversion

In [52]:
# pd.read_csv(
#     "../data/cases_daily_csv.csv", 
#     usecols=["date", "fips", "cases", "deaths"]
# ).to_parquet(
#     "../data/cases_dailt.parquet",
#     engine="pyarrow", 
#     compression="brotli"
# )

In [53]:
df_cases = pd.read_parquet("../data/cases_daily.parquet")
df_cases.fips = df_cases.fips.astype(str).str.zfill(5)
df_cases.head()

,date,fips,cases,deaths
0,2020-01-21,01001,0,0
1,2020-01-22,01001,0,0
2,2020-01-23,01001,0,0
3,2020-01-24,01001,0,0
4,2020-01-25,01001,0,0


In [63]:
df_cases["fips_state"] = df_cases.fips.str[0:2]
df_cases_grouped = df_cases \
    .drop(columns=["fips"]) \
    .groupby(["fips_state", "date"]) \
    .sum() \
    .reset_index() \
    .rename(columns={"fips_state": "fips"})

df_cases_grouped = df_cases_grouped.assign(fips=df_cases_grouped.fips.map(str) + "000")

df_cases_grouped.head()

,fips,date,cases,deaths
0,01000,2020-01-21,0,0
1,01000,2020-01-22,0,0
2,01000,2020-01-23,0,0
3,01000,2020-01-24,0,0
4,01000,2020-01-25,0,0


In [72]:
df_cases_total = df_cases_grouped \
    .groupby("date")[["cases", "deaths"]] \
    .sum() \
    .reset_index()

df_cases_total["fips"] = "00000"

df_cases_all = pd.concat([
    df_cases_total,
    df_cases_grouped,
    df_cases
]) \
    .loc[:, ["fips", "date", "cases", "deaths"]] \
    .sort_values(["fips", "date"])

df_cases_all = df_cases_all.loc[df_cases_all.fips < "57000", :].reset_index()

df_cases_all

,index,fips,date,cases,deaths
0,0,00000,2020-01-21,1,0
1,1,00000,2020-01-22,0,0
2,2,00000,2020-01-23,0,0
3,3,00000,2020-01-24,1,0
4,4,00000,2020-01-25,1,0
...,...,...,...,...,...
2756155,2711227,56045,2022-05-29,0,0
2756156,2711228,56045,2022-05-30,0,0
2756157,2711229,56045,2022-05-31,4,0
2756158,2711230,56045,2022-06-01,0,0


In [73]:
client["MDAProjectDatabase"]["daily_covid_cases"].insert_many(df_cases_all.to_dict("records"))

## Socio-economic data

In [ ]:
df_socio_economic = pd.read_csv("../data/non_epidemic_data.csv")
print(df_socio_economic.shape)
df_socio_economic.head()

In [ ]:
fipses_cases = pd.read_csv("../data/county_daily_csv.csv", usecols=["fips"]).fips.values

df_socio_economic = df_socio_economic.loc[df_socio_economic.fips.isin(fipses_cases), :]
df_socio_economic.shape

In [ ]:
client["MDAProjectDatabase"]["socio_economic_data"].insert_many(df_socio_economic.drop(columns=["area"]).to_dict("records"))

## FIPS dictionary

In [ ]:
df_fips = pd.read_csv("../data/fips.csv")

df_fips_states = df_fips.loc[df_fips.fips.values % 1000 == 0, :].copy()
df_fips_states.loc[:, "fips_state"] = [f"{int(fips / 1000):02d}" for fips in df_fips_states.fips]
df_fips_states = df_fips_states.drop(columns=["fips"]).rename(columns={"area": "state_name"})

df_fips = df_fips.loc[df_fips.fips.values % 1000 != 0, :]
df_fips.loc[:, "fips_state"] = [f"{fips:05d}"[:2] for fips in df_fips.fips]

df_fips = pd.merge(
    df_fips,
    df_fips_states,
    on="fips_state"
).drop(columns=["fips_state"])

df_fips.fips = df_fips.fips.astype(str).str.zfill(5)

df_fips.head()

In [ ]:
df_fips.head().to_dict("records")

In [ ]:
client["MDAProjectDatabase"]["fips_codes"].insert_many(df_fips.to_dict("records"))

## Time series forecasts

In [56]:
df_fcst = pd.read_csv("../data/country_level_cases_with_forecast.csv")
df_fcst.head()

,date,cases,Forecast 1,Forecast 2
0,2020-01-21,1.0,NaN,NaN
1,2020-01-22,0.0,NaN,NaN
2,2020-01-23,0.0,NaN,NaN
3,2020-01-24,1.0,NaN,NaN
4,2020-01-25,1.0,NaN,NaN


In [57]:
dict_list_no_nan = df_fcst.astype(object).where(df_fcst.notna(), None).to_dict("records")
dict_list_no_nan[0]

{'date': '2020-01-21', 'cases': 1.0, 'Forecast 1': None, 'Forecast 2': None}

In [58]:
db["country_level_cases_with_forecasts"].insert_many(dict_list_no_nan)

## Counterfactual models

In [4]:
# df_counterfactual1 = pd.read_csv("../data/counterfactual_results_model_1.csv")
# df_counterfactual1.head()

,date,variable,state_code,value
0,2020-09-30,mask_requirement,Mississippi,1513.580688
1,2020-09-30,no_mask_requirement,Mississippi,1513.463135
2,2020-10-01,mask_requirement,Mississippi,1536.039429
3,2020-10-01,no_mask_requirement,Mississippi,1529.435791
4,2020-10-02,mask_requirement,Michigan,984.426392


In [6]:
df_counterfactual2 = pd.read_csv("../data/counterfactual_results_model_2.csv").rename(columns={"state_code": "state_name"})
df_counterfactual2.head()

,date,variable,state_name,value
0,2020-09-30,mask_requirement,Mississippi,1555.972046
1,2020-09-30,no_mask_requirement,Mississippi,1573.680298
2,2020-10-01,mask_requirement,Mississippi,1587.778442
3,2020-10-01,no_mask_requirement,Mississippi,1636.114868
4,2020-10-02,mask_requirement,Michigan,1010.964966


In [7]:
db["counterfactual_model_results"].insert_many(df_counterfactual2.to_dict("records"))

## Vaccinations by state

In [75]:
df_abrev_state = pd.read_csv("../data/state_abrev_dict.csv")
df_abrev_state.fips_state = df_abrev_state.fips_state.astype(str).str.zfill(2)

df_vaccinations = pd.read_csv(
    "../data/vaccinations_by_state.csv", 
    dtype={"fully_vaccinated": int, "total_pop": int}
) \
    .rename(columns={"state_code": "abrev_state"})\
    .drop(columns=["pct_vaccinated"])

df_vaccinations = pd.merge(
    df_vaccinations,
    df_abrev_state,
    on="abrev_state",
    how="left"
)

df_vaccinations = pd.merge(
    df_vaccinations,
    pd.read_csv("../data/fips_states.csv", dtype={"fips_state": str}),
    on="fips_state",
    how="left"
)

df_vaccinations.state_name = df_vaccinations.state_name.str.title()
df_vaccinations = df_vaccinations \
    .loc[:, ["date", "state_name", "fully_vaccinated", "total_pop"]] \
    .dropna() \
    .reset_index(drop=True)

df_vaccinations_total = df_vaccinations \
    .groupby("date")[["fully_vaccinated", "total_pop"]] \
    .sum() \
    .reset_index()

df_vaccinations_total["state_name"] = "Total"

df_vaccinations = pd.concat([
    df_vaccinations_total,
    df_vaccinations
])

df_vaccinations

,date,fully_vaccinated,total_pop,state_name
0,2020-12-13,0,328239523,Total
1,2020-12-14,0,328239523,Total
2,2020-12-15,0,328239523,Total
3,2020-12-16,0,328239523,Total
4,2020-12-17,0,328239523,Total
...,...,...,...,...
27535,2022-06-01,287243,578759,Wyoming
27536,2022-06-02,287284,578759,Wyoming
27537,2022-06-03,287333,578759,Wyoming
27538,2022-06-04,287395,578759,Wyoming


In [76]:
db["vaccinations"].insert_many(df_vaccinations.to_dict("records"))

## Exp smoothing params

In [32]:
df_exp_smoothing_best_params = pd.read_csv("../data/best_exp_smoothing_params.csv")
df_exp_smoothing_best_params

,trend,damped_trend,seasonal,initialization_method,use_boxcox,MAPE
0,add,True,add,estimated,True,0.346966
1,add,True,add,heuristic,True,0.346966
2,add,False,mul,estimated,False,0.361512
3,add,False,mul,heuristic,False,0.361512
4,add,True,mul,estimated,False,0.369168


In [34]:
db["df_exp_smoothing_best_params"].insert_many(df_exp_smoothing_best_params.to_dict("records"))